In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np

In [2]:
from tcc_video_utils import compute_alignment_loss

In [3]:
embs = torch.randn(16, 10, 512)
batch_size = 2

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
min_chunk_length = 3

num_steps = embs.shape[1]
disc = round(np.sqrt(1 + 8 * num_steps))
assert disc * disc == 1 + 8 * num_steps, "cannot find integer num_steps such that seq_lens * (seq_lens - 1) / 2, please provide your own steps and seq_lens."

video_lengths = (1 + disc) // 2 + min_chunk_length
seq_lens = torch.tensor(video_lengths, device=device).unsqueeze(0).repeat([batch_size])

steps_list = []
for i in range(video_lengths):
    for j in range(i + 1 + min_chunk_length, video_lengths):
        steps_list.append([i, j])
steps = torch.tensor(steps_list, device=device).unsqueeze(0).repeat([batch_size, 1, 1])

In [5]:
loss = compute_alignment_loss(embs, steps, seq_lens)
loss

torch.Size([20, 3])


tensor(1.9409, device='cuda:0')

In [53]:
all_start = []
all_end = []

for _ in range(1000):
    embs = torch.randn(16, 10, 512)
    batch_size = 20
    start_loss, end_loss = compute_alignment_loss(embs)
    all_start.append(start_loss)
    all_end.append(end_loss)

torch.stack(all_start).mean(), torch.stack(all_end).mean()

(tensor(0.9625, device='cuda:0'), tensor(0.9754, device='cuda:0'))

In [6]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5])
torch.stack([a, b])

RuntimeError: stack expects each tensor to be equal size, but got [3] at entry 0 and [2] at entry 1

In [5]:
tmp = torch.tensor([[3, 4], [3, 4], [3, 4], [2, 4], [2, 4], [2, 4]])
tmp.reshape([2, 3, 2])

tensor([[[3, 4],
         [3, 4],
         [3, 4]],

        [[2, 4],
         [2, 4],
         [2, 4]]])